In [5]:
#@title Install and import necessary libraries
%pip install -qU "tool-parse[langchain]" langchain-groq langgraph duckduckgo-search

import typing as t

from duckduckgo_search import DDGS

from langchain_groq.chat_models import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from tool_parse.integrations.langchain import ExtendedStructuredTool, patch_chat_model

#### Define tools

In [17]:
def search_text(
    text: str,
    *,
    safe_search: bool = True,
    backend: t.Literal["api", "html", "lite"] = "api",
    max_results: int = 1,
):
    """
    Search for text in the web.
    :param text: Text to search for.
    :param safe_search: If True, enable safe search.
    :param backend: Backend to use for retrieving results.
    :param max_results: Max results to return.
    """
    return DDGS().text(
        keywords=text,
        safesearch="on" if safe_search else "off",
        backend=backend,
        max_results=max_results,
    )

class ProductInfo(t.NamedTuple):  # Can be t.TypedDict or pydantic.BaseModel
    """
    Information about the product.
    :param name: Name of the product.
    :param price: Price of the product.
    :param in_stock: If the product is in stock.
    """

    name: str
    price: float
    in_stock: bool = False

tools = [
    ExtendedStructuredTool(func=search_text),
    ExtendedStructuredTool(func=ProductInfo, name="product_info"),
]
# OR # tools = ExtendedStructuredTool.from_objects(search_text, ProductInfo)
tools

[ExtendedStructuredTool(name='search_text', description='Search for text in the web.', func=<function search_text at 0x0000012F2A321120>),
 ExtendedStructuredTool(name='product_info', description='Information about the product.', func=<class '__main__.ProductInfo'>)]

#### Patch the langchain chat model

In [4]:
# Set groq api key (https://console.groq.com/keys)

import os, getpass

def set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

set_env("GROQ_API_KEY")

In [18]:
# Patch the instance
model = patch_chat_model(ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model="llama3-70b-8192"))
# OR Patch the class and then instantiate it
model = patch_chat_model(ChatGroq)(api_key=os.getenv("GROQ_API_KEY"), model="llama3-70b-8192")

#### Create a react agent

In [19]:
agent_executor = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "id123"}}

def query_agent(__query: str):
    for chunk in agent_executor.stream({"messages": [HumanMessage(content=__query)]}, config):
        message = list(chunk.values())[0]["messages"][0]
        if isinstance(message, AIMessage):
            print("[AGENT]")
            if message.tool_calls:
                print("Calling tool:")
                metadata = message.tool_calls[0]
                print(f"name={metadata['name']!r}")
                print(f"arguments={metadata['args']}")
            else:
                print(message.content)
        elif isinstance(message, ToolMessage):
            print(f"[{message.name!r} TOOL OUTPUT]")
            print(message.content)

        print("---" * 20)

In [20]:
query_agent("Search 5 sources for langgraph docs using lite backend")

[AGENT]
Calling tool:
name='search_text'
arguments={'backend': 'lite', 'max_results': 5, 'text': 'langgraph docs'}
------------------------------------------------------------


C:\Users\synac\AppData\Local\Temp\ipykernel_7024\857887115.py:15: UserWarning: lxml is not installed. Using backend='api'.
  return DDGS().text(


['search_text' TOOL OUTPUT]
[{"title": "LangGraph - LangChain", "href": "https://www.langchain.com/langgraph", "body": "LangGraph is a framework for building stateful, multi-agent systems with LLMs. Learn how to use LangGraph with Python or JavaScript, see conceptual guides, and access LangGraph Cloud for deployment and scaling."}, {"title": "️LangGraph - GitHub Pages", "href": "https://langchain-ai.github.io/langgraph/", "body": "LangGraph is a framework for creating stateful, multi-actor applications with LLMs, using cycles, controllability, and persistence. Learn how to use LangGraph with examples, features, and integration with LangChain and LangSmith."}, {"title": "️LangGraph.js - GitHub Pages", "href": "https://langchain-ai.github.io/langgraphjs/", "body": "LangGraph.js is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and

In [21]:
query_agent("Parse the product: RTX 4900, priced at $3.5k, is in stock.")

[AGENT]
Calling tool:
name='product_info'
arguments={'in_stock': True, 'name': 'RTX 4900', 'price': 3500}
------------------------------------------------------------
['product_info' TOOL OUTPUT]
["RTX 4900", 3500.0, true]
------------------------------------------------------------
[AGENT]
Here is the product information:

The product is RTX 4900, priced at $3500.00, and it is in stock.
------------------------------------------------------------
